In [ ]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer-test.microsoft.com/stac",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
time_range = "2022-01-01/2022-12-31"
search = catalog.search(collections=["naip"], datetime=time_range)
items = search.item_collection()
len(items)

In [ ]:
import contextily
import geopandas

df = geopandas.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")

ax = df[["geometry", "datetime"]].plot(
    facecolor="none", figsize=(12, 8)
)
contextily.add_basemap(
    ax, crs=df.crs.to_string(), source=contextily.providers.Esri.NatGeoWorldMap
)

In [ ]:
df.head()
summary_table = df.groupby("naip:state").size().reset_index(name="Count")
summary_table

In [ ]:
import matplotlib.pyplot as plt
# One day Alaska will be included in NAIP
states = [
    {
        "code": "hi",
        "name": "Hawaii"
    },
    {
        "code": "pr",
        "name": "Puerto Rico"
    },
    {
        "code": "ak",
        "name": "Alaska"
    },
    {
        "code": "vi",
        "name": "Virgin Islands"
    },
]

fig, axs = plt.subplots(len(states), 1, figsize=(12, 8))

for idx, state in enumerate(states):
    stateDf = df[df["naip:state"] == state["code"]]
    if stateDf.empty:
        continue
    merged_polygon = stateDf["geometry"].unary_union
    bounding_box = merged_polygon.bounds
    stateDf.plot(ax=axs[idx])  # f"{state} {bounding_box}")
    axs[idx].set_title(f"{state['name']} {bounding_box}")

plt.tight_layout()
plt.show()
